# PROJECT 1 (PART 1)

## Exploratory data analysis and preprocessing

The **exploratory data analysis** is the statistical treatment to which the samples collected during a research process in any scientific field are subjected.
For greater speed and accuracy, the entire process is usually carried out by computer, with specific applications for statistical treatment.

### Application to Data Mining

In **data mining**, although not mandatory, it is a good practice to analyze the data you will be working with in order to observe its main characteristics in order to get an idea of the structure of the data set, and identify the target variable and possible modeling techniques.

**Key steps:**
- *Transform the data*: It helps us know what to do with null, missing values, or atypical data. In addition, it establishes if there is a need to reduce the dimensionality of data.
- *Visualize*: Use some tool to make a graphical representation of the data, for example, R, Jupyter notebook, Google Colab, etc.
- *Analyze and interpret*: Analyze and interpret the data through different visualizations.
- *Document*: Document all the graphs and statistics generated.

This process is also helpful when reviewing the data description to understand the meaning of each characteristic.

There are several activities in doing an exploratory data analysis but in terms of data mining the key points to be made are:

- Description of the data structure.
- Identification of missing data.
- Detection of outliers.
- Identification of relationships between variable pairs.

The **goal of this project** is to learn how to do data exploration. In this case, data from **New York City Yellow Taxis** is used.

At the end of the notebook, you should be able to answer the following question:


## How has covid affected the use of taxis in New York?

Some of the questions you will ask yourselves throughout the notebook are:
- How has covid changed the use of taxis in NYC?
- What pick-up distribution do the taxis follow and what distance / duration do they take?
- What are the areas where taxis are picked up the least? And where else do people go?
- What are the most usual times?
- Which days of the week and month are used the most? Possible reasons?

**Install and import the required libraries**

In [ ]:
# Install libraries
!pip install pyarrow
!pip install pyshp
!pip install shapely==1.7.1
!pip install descartes

In [ ]:
# Import libraries - data processing
import pandas as pd
import numpy as np
import pyarrow.parquet as pq

In [ ]:
# Import libraries -  manipulation of files and downloads
import urllib.request
import zipfile
import os
from tqdm.notebook import tqdm

In [ ]:
# Import libraries - visualitzation
import shapefile
from shapely.geometry import Polygon
from descartes.patch import PolygonPatch
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Global variables
YEARS = [2018, 2019, 2020, 2021, 2022]

In [ ]:
# Definition of the data path in Google Drive
taxi_data_folder = '/content/drive/MyDrive/data'

First of all, you need to download the data:

https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page

In [ ]:
# Mount Google Drive (only required if running from Google Colab)
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# Creation of the data folder if it does not exist
if not os.path.exists(taxi_data_folder):
    os.makedirs(taxi_data_folder, exist_ok=True)

In [ ]:
# Download the Trip Record Data
for year in tqdm(YEARS):
    year_folder = f'{taxi_data_folder}/{year}'
    if not os.path.exists(year_folder):
        os.makedirs(year_folder, exist_ok=True)
        for month in tqdm(range(1, 13)):
            month_file_path = f'{year_folder}/{month:02d}.parquet'
            if not os.path.exists(month_file_path):
                urllib.request.urlretrieve(f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year}-{month:02d}.parquet', month_file_path)


## 01. Data cleaning

In order to have clean and useful data, it is necessary to delete all those rows that contain corrupt information:
- The pick-up is after the drop-off.
- Dates are imported by months and years. Are the dates correct?
- Traveling with zero passengers?
- Do you travel very long or particularly short?
- Negative payments.

**Data Dictionary**

Only the next columns are needed:

- *tpep_pickup_datetime*: The date and time when the meter was engaged.
- *tpep_dropoff_datetime*: The date and time when the meter was disengaged.
- *Passenger_count*: The number of passengers in the vehicle. (This is a driver-entered value)
- *Trip_distance*: The elapsed trip distance in miles reported by the taximeter.
- *PULocationID*: TLC Taxi Zone in which the taximeter was engaged
- *DOLocationID*: TLC Taxi Zone in which the taximeter was disengaged
- *Payment_type*: A numeric code signifying how the passenger paid for the trip.
    - 1= Credit card
    - 2= Cash
    - 3= No charge
    - 4= Dispute
    - 5= Unknown
    - 6= Voided trip
- *Fare_amount*: The time-and-distance fare calculated by the meter.
- *Total_amount*: The total amount charged to passengers. Does not include cash tips.

**Remarks:**

- To speed up the calculations and reduce the computation time, do a uniform sampling of the data (a sample out of 1000).
- Datetime columns are *to_datetime* series (help to search functions)

In [ ]:
def load_table(year, month):
    """
    Function that reads the downloaded data and converts it to a DataFrame
    """

    file_path = os.path.join(taxi_data_folder, str(year), f'{str(month).zfill(2)}.parquet')

    return pq.read_table(file_path).to_pandas()

In [ ]:
required_data = ['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance',
                 'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'total_amount']

def clean_data(data, year, month, sampling = 1000):
    """
    Cleans the New York taxi data for a specific month and year.

    Parameters:
    - data: DataFrame containing the taxi data.
    - year: Year to filter on.
    - month: Month to filter on.
    - sampling

    Returns:
    - A cleaned DataFrame.
    """
    # Filter data based on year and month
    data = data[(data['tpep_pickup_datetime'].dt.year == year) & (data['tpep_pickup_datetime'].dt.month == month)]

    # Sample the data
    data = data.sample(n=sampling, random_state=42)

    # Select only the required columns
    data = data[required_data]


    return data

In [ ]:
df = load_table(2018,12)

In [ ]:
clean_data = clean_data(df, 2019, 12, 1000)

In [ ]:
clean_data

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,PULocationID,DOLocationID,payment_type,fare_amount,total_amount
6586714,2019-12-30 14:06:20,2019-12-30 14:39:46,1.0,8.60,132,93,4,-28.5,-29.30
6750903,2019-12-31 14:16:07,2019-12-31 14:26:04,1.0,1.75,236,163,1,9.0,14.76
6253770,2019-12-28 15:20:05,2019-12-28 15:26:54,1.0,2.01,237,75,1,8.0,13.56
3004792,2019-12-13 10:19:51,2019-12-13 10:35:09,6.0,5.10,162,88,1,18.0,23.43
3124249,2019-12-13 19:33:42,2019-12-13 19:57:29,1.0,2.43,100,4,1,15.0,21.23
...,...,...,...,...,...,...,...,...,...
5007002,2019-12-20 23:55:53,2019-12-21 00:14:11,1.0,3.30,249,229,1,14.0,21.35
1968391,2019-12-09 09:37:47,2019-12-09 09:37:49,1.0,0.00,132,264,1,65.0,78.36
2352508,2019-12-10 21:34:17,2019-12-10 21:40:40,1.0,0.50,231,231,1,5.5,10.30
5270246,2019-12-22 07:03:02,2019-12-22 07:06:30,1.0,0.50,256,255,1,4.5,5.30


In the ***post_processing*** function you can add all information you need in order to perform the necessary exploration.

Particularly, this function should achieve the following:

1. Extract the month, week number, day of the week, hour, and day of the year from both the tpep_pickup_datetime and tpep_dropoff_datetime columns. This will give us a finer understanding of the patterns within our data. For clarity, your derived columns should be named as pickup_month, **pickup_week**, **pickup_day**, **pickup_hour**, **pickup_dayofyear**, and similarly for the drop-off attributes.

2. Calculate the trip duration in hours. This will help us understand the lengths of the trips people take. You can name this column as **trip_duration**.

3. Convert the trip_distance from miles to kilometers. Please ensure you create a new column for this, named **trip_distance**.

In [ ]:
def post_processing(data):

    """
    Function to implement any type of post-processing required.
    """
    data["pickup_hour"]=data["tpep_pickup_datetime"].dt.hour
    data["pickup_day"]=data["tpep_pickup_datetime"].dt.day
    data["pickup_dayofyear"]=data["tpep_pickup_datetime"].dt.dayofyear
    data["pickup_week"]=data["tpep_pickup_datetime"].dt.week

    data["dropoff_hour"]=data["tpep_dropoff_datetime"].dt.hour
    data["dropoff_day"]=data["tpep_dropoff_datetime"].dt.day
    data["dropoff_dayofyear"]=data["tpep_dropoff_datetime"].dt.dayofyear
    data["dropoff_week"]=data["tpep_dropoff_datetime"].dt.week

    # calculate trip duration
    data["trip_duration"]=data["tpep_dropoff_datetime"]-data["tpep_pickup_datetime"]

    # convert trip_distance to km, create a new column
    data["trip_distance"]=data["trip_distance"]*1.60934

    return data

Create a new dataset that contains all the information for the years: 2019, 2020, and 2021.

Remember that in order to reduce the memory required, you can take a subsample of the data.

In [ ]:
df = pd.concat([clean_data(load_table(year, month), year, month) for year in tqdm(YEARS) for month in tqdm(range(1, 13), leave = False)])

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

In [ ]:
df

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,PULocationID,DOLocationID,payment_type,fare_amount,total_amount
3262878,2018-01-13 15:07:28,2018-01-13 15:16:35,1.0,1.90,107,237,4,9.0,9.80
3564551,2018-01-14 13:24:40,2018-01-14 13:40:32,1.0,1.57,229,230,2,11.0,11.80
3718542,2018-01-14 23:42:41,2018-01-14 23:47:32,1.0,2.00,74,263,2,7.0,8.30
7194455,2018-01-26 18:48:56,2018-01-26 18:58:56,2.0,1.63,233,141,1,8.5,11.55
5077448,2018-01-19 16:03:05,2018-01-19 16:07:35,1.0,0.60,161,161,1,5.0,8.16
...,...,...,...,...,...,...,...,...,...
440153,2022-12-04 15:44:59,2022-12-04 15:57:29,1.0,1.98,107,144,2,10.0,13.30
2080396,2022-12-18 14:49:55,2022-12-18 15:03:21,2.0,1.56,68,114,1,9.5,15.36
1695583,2022-12-15 10:39:58,2022-12-15 11:41:04,1.0,8.37,138,230,1,41.0,54.66
1927920,2022-12-17 03:38:59,2022-12-17 04:05:58,2.0,7.13,148,151,1,23.5,32.76


In [ ]:
df.columns

Index(['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count',
       'trip_distance', 'PULocationID', 'DOLocationID', 'payment_type',
       'fare_amount', 'total_amount'],
      dtype='object')

In [ ]:
df = post_processing(df)
df.shape

<ipython-input-49-6ed57b9c279d>:9: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  data["pickup_week"]=data["tpep_pickup_datetime"].dt.week
<ipython-input-49-6ed57b9c279d>:14: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  data["dropoff_week"]=data["tpep_dropoff_datetime"].dt.week


(60000, 18)

In [ ]:
df.head(2)

In [ ]:
# Save clean df
df.to_csv(f'{taxi_data_folder}/clean_data.csv', index = False)

## Pandas Sheet

**Pandas Concepts and Functions**

1. **DataFrame**:

  - A DataFrame is the primary data structure in Pandas, similar to an Excel worksheet.
  - It's a 2-dimensional labeled data structure with rows and columns.

2. **DataFrame Indexing**:

  - This refers to selecting specific rows and/or columns from a DataFrame.
  - For example, **data['column_name']** would select a specific column from the DataFrame.
  - Other way to do it is using **'.loc'**. It is a label-based data selection method which means that we have to pass the name of the row or/and column which we want to select: **data.loc[row_condition, ['columns_name']]**.

3. **Datetime Properties**:

  - Datetime in Pandas allows us to work with date and time data.
  With **'.dt'** accessor, we can access properties like year, month, day, etc.
  - Example: **data['date_column'].dt.year** would extract the year from each date in the column.

4. **Drop NaN Values**:

  - NaN stands for "Not a Number" and indicates a missing value.
  - The '**dropna()**' function is used to remove missing values.
  - Example: **'data.dropna()'** would drop any row in the DataFrame that contains at least one NaN value.

5. **Filtering**:

  - This refers to selecting specific rows in a DataFrame based on some condition.
  - Example: data[data['age'] > 18] would select rows where the age column's value is greater than 18.

6. **Reset Index**:

  - Sometimes, after filtering or other operations, the index of a DataFrame can become out of order. Resetting it creates a new, orderly index.
  - **'reset_index(drop=True)'** will reset the index and drop the old index values.

7. **Boolean Indexing**:

  - It's a filtering method used in Pandas where rows are selected based on True/False values, typically from a condition applied to a column.
  - Example: If condition is an array of boolean values, then '**data[condition]**' will return rows where the condition is True.

8. **Column Creation/Modification**:

  - In Pandas, you can easily create a new column or modify an existing one.
  - Example: **data['new_column'] = data['old_column'] * 2** would create a new column with values doubled from the old column.